In [35]:
import os

In [38]:
%pwd

'C:\\Users\\91985\\Desktop\\123\\Projects\\ML\\news-app'

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [70]:
from src.core.constants import *
from src.core.utils.common import read_yaml, create_directories

In [95]:
%pwd

'C:\\Users\\91985\\Desktop\\123\\Projects\\ML\\news-app'

In [91]:
from pathlib import Path
CONFIG_FILE_PATH = Path("config\config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [92]:
import os
import urllib.request as request
import zipfile
from src.core.logging import logger
from src.core.utils.common import get_size

In [93]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [94]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2024-03-13 01:03:51,530 - INFO - yaml file: config\config.yaml loaded successfully
2024-03-13 01:03:51,532 - INFO - yaml file: params.yaml loaded successfully
2024-03-13 01:03:51,534 - INFO - created directory at: artifacts
2024-03-13 01:03:51,561 - INFO - created directory at: artifacts/data_ingestion
2024-03-13 01:03:57,301 - INFO - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5D8C:2361DE:2D9C1E:3C0BA4:65F0AE26
Accept-Ranges: bytes
Date: Tue, 12 Mar 2024 19:34:01 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10221-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1710272041.79